<h1 align=center><font size = 5>Toronto neighborhoods clustering</font></h1>

## Part 1


In [1]:
!pip install BeautifulSoup4
!pip install lxml
!pip install requests

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
#convert the link into a txet
link = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
#Creates a soup object
soup = BeautifulSoup(link, 'lxml')

In [ ]:
#using soup object cretes a list ftom the "wikitable"
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

In [ ]:
#convert list into Pandas DataFrame
dbtoronto = pd.DataFrame(data = data,columns = columns)
dbtoronto.head()

In [ ]:
#Erases the rows with "Not assigned" in the Borough column
dbtoronto=dbtoronto[dbtoronto['Borough']!='Not assigned']

In [ ]:
#Combines neighborhoods in the same postal code area
dbtoronto["Neighborhood"] = dbtoronto.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))
#For neighborhoods thath have Borough but "Not assigned" Neighborhood, the Borough will be also set as the Neighborhood
dbtoronto['Neighborhood'].replace("Not assigned", dbtoronto["Borough"],inplace=True)

In [ ]:
#Change the index name to the postal code
dbtoronto=dbtoronto.set_index('Postal Code')

In [ ]:
dbtoronto.shape


In [ ]:
dbtoronto


## Part 2


In [ ]:
!wget -q -O 'torontocoordinates.csv' http://cocl.us/Geospatial_data
print("Data downloaded")

In [ ]:
llcoord=pd.read_csv("torontocoordinates.csv")
llcoord.head()

In [ ]:
#Changes the index of llcoord to the Postal Code
llcoord=llcoord.set_index('Postal Code')
llcoord.head()

In [ ]:
#Joins both data bases with respect to the index
dbtoronto=dbtoronto.join(llcoord)
#Resets the index
dbtoronto=dbtoronto.reset_index()
dbtoronto.head()